# Grade prediction with BERT embedding

In this notebook, we will use pre-trained deep learning model to process some text. We will then use the output of that model to classify the reviews we scrapped in previous courses. We will try to predict wether it was positive or negative. 

Under the hood, the model is actually made up of two model.


1.   DistilBERT processes the sentence and passes along some information it extracted from it on to the next model. DistilBERT is a smaller version of BERT developed and open sourced by the team at HuggingFace. It’s a lighter and faster version of BERT that roughly matches its performance.
2.   The next model, a basic Logistic Regression model from scikit learn will identify if the review was positive or negative



## Installing the transformers library

Let's start by installing the huggingface transformers library so we can load our deep learning NLP model.

In [1]:
!pip install transformers

In [147]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import nltk
import re
import itertools
import unidecode
import tensorflow as tf
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

## Importing the dataset

We'll import the dataset as we did in previous notebooks:

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
dirpath = "drive" 
os.listdir(dirpath)

Mounted at /content/drive


['Othercomputers',
 '.file-revisions-by-id',
 'MyDrive',
 '.shortcut-targets-by-id',
 '.Trash-0']

In [186]:
import os
# IMPORT YOUR REVIEWS FILE
reviews = pd.read_csv("corpus_total.csv")

In [187]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            200 non-null    int64 
 1   text             200 non-null    object
 2   grade            200 non-null    int64 
 3   date             200 non-null    object
 4   company          200 non-null    object
 5   n_words          200 non-null    int64 
 6   clean_text       200 non-null    object
 7   tokenized_text   200 non-null    object
 8   bigrams          200 non-null    object
 9   trigrams         200 non-null    object
 10  wordDict         200 non-null    object
 11  tfBOW            200 non-null    object
 12  tfIDF            200 non-null    object
 13  lemmatized_text  200 non-null    object
 14  stemmed_text     200 non-null    object
dtypes: int64(3), object(12)
memory usage: 23.6+ KB


In the first place we should determine what are we considering a positive review. In this case we shall consider as positive every comment that had 4 or 5 as grade and negative otherwise.

In [197]:
# ASSIGN A LABEL TO EACH REVIEW TO DIFFERENTIATE POSITIVE AND NEGATIVE REVIEWS
reviews["sentiment"] = reviews.grade.isin([4,5])
# REMOVE REVIEWS WITH NULL VALUES
print("Number of nan reviews: ", reviews.text.isna().sum())
print("Number of Empty Reviews: ", sum([len(i) == 0 for i in reviews.text]))


Number of nan reviews:  0
Number of Empty Reviews:  0


For computational purposes, we'll only use 1000 sentences which have less than 100 words in it.

In [189]:

def clean_text(text):
    # Remove "\n" characters
    text = re.sub(r'\n', ' ', text)
    # Remove emojis and other non-letter characters (but keep punctuation)
    text = re.sub(r'[^\w\s\d\.,;:\?!\'\"\(\)\-\$]+', ' ', text)
    # Remove excess white space
    text = re.sub(r'\s+', ' ', text).strip()
    return text

#Keep punctuation
reviews["verbatim"] = reviews.text.apply(clean_text)

In [190]:
# Filter the reviews
def split_reviews_per_sentence(reviews, col ):
    reviews["review_sentences"] = reviews[col].apply(
        lambda rvw: nltk.sent_tokenize(rvw)
    )
    return reviews

reviews = split_reviews_per_sentence(reviews, "verbatim")


In [191]:
reviews.review_sentences[0]

['Service électricité très bien mais si Service électricité très bien mais si vous avez un surplus de règlement ça se complique.',
 "Le mot le plus employé par les services patience C'est pas un service comptable qu ils ont à priori mais plutôt une usine à gaz... Franchement j'ai les nerfs de faire la trésorerie de Total"]

In [195]:
#Save in two variables the comments and the labels

sentences = list(itertools.chain(*reviews["review_sentences"]))
print(f"Number of total sentences : {len(sentences)}")

sentence_lengths = [len(i.split(" ")) for i in sentences]
cond  = [i > 100 for i in sentence_lengths]

long_sentences = [x for x, m in zip(sentence_lengths, cond) if m]
print("Number of sentences longer than 100 words: ", len(long_sentences))
labels = [int(i) for i in reviews.sentiment]


Number of total sentences : 732
Number of sentences longer than 100 words:  3


There are 3 sentences longer than 100 words. Since our corpus text which is the last 200 reviews for Total Energies contain less than 1000 sentences, we will keep these 3 sentences.

## Loading the Pre-trained BERT model
Let's now load a pre-trained BERT model.

In [68]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Right now, the variable model holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.

# Model 1: Preparing the Dataset
Before we can hand our sentences to BERT, we need to do some minimal processing to put them in the format it requires.

### Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

In [155]:
# Apply tokenization
def clean_sentence(sentence, stopwords=False):
    sentence = sentence.lower().strip() #lowercase
    sentence = unidecode.unidecode(sentence) #remove accents from letters
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence) #remove non-alphanumeric characters
    sentence = re.sub(r"\d+", "", sentence) #remove numbers
    sentence = re.sub(r"\s+"," ",sentence) #remove whitespace
    sentence = sentence.rstrip().lstrip() #remove space at the beginning and end if it exists
    
    return sentence

sentences = [clean_sentence(i) for i in sentences]
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters=' ', char_level=False)
tokenizer.fit_on_texts(sentences)

tokenized = [i.split(" ") for i in sentences]

We created a 

In [154]:
sequences = tokenizer.texts_to_sequences(sentences)
print(reviews["verbatim"][1][0], sentences[0], sequences[0], sep="\n\n")

H

service electricite tres bien mais si service electricite tres bien mais si vous avez un surplus de reglement ca se complique

[15, 95, 26, 45, 40, 56, 15, 95, 26, 45, 40, 56, 23, 313, 9, 651, 1, 652, 73, 65, 653]


In [185]:
len(sentences)

732

In [182]:
len(labels)

200

### Padding
After tokenization, tokenized is a list of sentences -- each sentences is represented as a list of tokens. We want BERT to process our examples all at once (as one batch). It's just faster that way. For that reason, we need to pad all lists to the same size, so we can represent the input as one 2-d array, rather than a list of lists (of different lengths).


In [172]:
# Perform the padding

max_len = 0
for i in tokenized:
    if len(i) > max_len:
        max_len = len(i)
max_len
padded = [token + [0] * (max_len - len(token)) for token in sequences]
print(padded[0])

#check if all sizes are 113
(pd.Series([len(pad) for pad in padded]) == max_len).all()

[15, 95, 26, 45, 40, 56, 15, 95, 26, 45, 40, 56, 23, 313, 9, 651, 1, 652, 73, 65, 653, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


True

Our dataset is now in the padded variable, we can view its dimensions below:

In [173]:
padded = np.array(padded)

### Masking
If we directly send padded to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [174]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(732, 113)

## Model 1: And Now, Deep Learning!
Now that we have our model and inputs ready, let's run our model! 

The model() function runs our sentences through BERT. The results of the processing will be returned into last_hidden_states.

In [168]:
max(tokenizer.index_word.keys())

2352

In [175]:
padded

array([[  15,   95,   26, ...,    0,    0,    0],
       [   5, 1043,    5, ...,    0,    0,    0],
       [ 269,  269,   56, ...,    0,    0,    0],
       ...,
       [  45,  613,    0, ...,    0,    0,    0],
       [  97,   92,    0, ...,    0,    0,    0],
       [  97,   92,    0, ...,    0,    0,    0]])

In [176]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

Let's slice only the part of the output that we need. That is the output corresponding the first token of each sentence. The way BERT does sentence classification, is that it adds a token called [CLS] (for classification) at the beginning of every sentence. The output corresponding to that token can be thought of as an embedding for the entire sentence.

We'll save those in the features variable, as they'll serve as the features to our logitics regression model.



In [177]:
features = last_hidden_states[0][:,0,:].numpy()

In [179]:
features.shape

(732, 768)

## Model 2: Train/Test Split
Let's now split our datset into a training set and testing set (even though we're using 1,000 sentences from the reviews training set).

In [181]:
# Split the data in train and test with the train_test_split function
Xtrain, ytrain = train_test_split(features)



## [Bonus] Grid Search for Parameters
We can dive into Logistic regression directly with the Scikit Learn default parameters, but sometimes it's worth searching for the best value of the C parameter, which determines regularization strength.

We now train the LogisticRegression model. If you've chosen to do the gridsearch, you can plug the value of C into the model declaration (e.g. LogisticRegression(C=5.2)).

In [ ]:
# Fit the logistic regression model

## Evaluating Model 2
So how well does our model do in classifying sentences? One way is to check the accuracy against the testing dataset:

In [ ]:
# Evaluate the score of the model

How good is this score? What can we compare it against? Let's first look at a dummy classifier:

In [ ]:

from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

# COmpare your model with a dummy classifier

So our model clearly does better than a dummy classifier